In [1]:
print("tesy")

tesy


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset
import torchaudio
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from einops import rearrange
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Add at the end of each epoch in both training functions
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Environment Sound Dataset classes (will be populated from the dataset directory)
ENV_SOUND_CLASSES = []

# Audio processing parameters
SAMPLE_RATE = 16000
DURATION = 10  # seconds
SEQUENCE_LENGTH = SAMPLE_RATE * DURATION  # 160,000 samples for 10 seconds

# Model parameters
LATENT_DIM = 384  # Embedding dimension
FEATURE_DIM = 512  # Feature dimension for CNN frontend
NUM_LAYERS = 6     # Encoder layers
NUM_DECODER_LAYERS = 8  # Decoder layers
NUM_HEADS = 8     # Attention heads
MLP_RATIO = 4     # Ratio for MLP hidden dim
NUM_CLASSES = len(ENV_SOUND_CLASSES)  # Number of classes for classification
CNN_KERNEL_SIZE = 400   # Kernel size for CNN front-end (25ms at 16kHz)
CNN_STRIDE = 160        # Stride for CNN front-end (10ms at 16kHz)
NUM_TOKENS = SEQUENCE_LENGTH // CNN_STRIDE  # Number of tokens after CNN

# Training parameters
BATCH_SIZE = 8
NUM_EPOCHS_PRETRAIN = 24
NUM_EPOCHS_FINETUNE = 60
LEARNING_RATE_PRETRAIN = 2e-4
LEARNING_RATE_FINETUNE = 5e-5
WEIGHT_DECAY = 0.05
MASK_RATIO_FINETUNE = 0.1  # 10% regularized patch masking for fine-tuning

# Path to dataset
DATA_PATH = "/kaggle/input/environment-sound/Environment_Sound"  # Correct path to Environment Sound dataset

# Define dataset class for raw audio
class EnvironmentSoundDataset(Dataset):
    def __init__(self, root_dir):
        """
        Args:
            root_dir (string): Dataset root directory with class-named subfolders.
        """
        self.root_dir = root_dir
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        self.audio_files = []
        self.labels = []

        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for fname in os.listdir(class_path):
                if fname.endswith(".wav"):
                    self.audio_files.append(os.path.join(class_path, fname))
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_path = self.audio_files[idx]
        label = self.labels[idx]

        # Load audio
        waveform, sample_rate = torchaudio.load(audio_path)

        # Convert to mono
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # Resample if needed
        if sample_rate != SAMPLE_RATE:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=SAMPLE_RATE)
            waveform = resampler(waveform)

        # Pad/crop to fixed duration
        target_length = SAMPLE_RATE * DURATION
        current_length = waveform.shape[1]
        if current_length < target_length:
            padding = target_length - current_length
            waveform = F.pad(waveform, (0, padding))
        elif current_length > target_length:
            start = np.random.randint(0, current_length - target_length)
            waveform = waveform[:, start:start + target_length]

        # Normalize waveform
        waveform = (waveform - waveform.mean()) / (waveform.std() + 1e-6)

        return waveform, label

# 1D CNN Feature Extractor (similar to wav2Vec approach)
class CNNFeatureExtractor(nn.Module):
    def __init__(self, in_channels=1, out_channels=FEATURE_DIM, kernel_size=CNN_KERNEL_SIZE, 
                 stride=CNN_STRIDE, embed_dim=LATENT_DIM):
        super().__init__()
        
        # Initial conv layer to handle the raw audio - same as wav2vec approach
        self.initial_conv = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size//2),
            nn.BatchNorm1d(out_channels),
            nn.GELU(),
        )
        
        # Multi-scale 1D CNN feature extractor
        self.conv_blocks = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU(),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU(),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU()
            ),
            nn.Sequential(
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU(),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.GELU()
            )
        ])
        
        # Projection to embedding dimension
        self.projection = nn.Linear(out_channels, embed_dim)
        self.layer_norm = nn.LayerNorm(embed_dim)
        
        # Calculate number of tokens after convolution (with padding)
        self.num_tokens = (SEQUENCE_LENGTH + 2*(kernel_size//2) - kernel_size) // stride + 1
        
    def forward(self, x):
        # Input: (B, C, L) - [batch, channels, length]
        
        # Apply initial convolution
        x = self.initial_conv(x)
        
        # Apply additional CNN blocks
        for conv_block in self.conv_blocks:
            x = conv_block(x)
        
        # Transpose: (B, C, L) -> (B, L, C)
        x = x.transpose(1, 2)
        
        # Project to embedding dimension
        x = self.projection(x)
        x = self.layer_norm(x)
        
        # Output: (B, L, D) - [batch, sequence_length, embed_dim]
        return x

# Define the MLP block
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features * MLP_RATIO
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# Define the transformer encoder block
class TransformerEncoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, dropout=attn_drop, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(in_features=dim, hidden_features=int(dim * mlp_ratio), drop=drop)
        
    def forward(self, x, mask=None):
        # Self-attention with residual connection
        if mask is not None:
            # Apply mask to attention if provided
            attn_output, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x), 
                                     key_padding_mask=mask)
        else:
            attn_output, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))
        
        x = x + attn_output
        
        # MLP with residual connection
        x = x + self.mlp(self.norm2(x))
        return x

# Define the Audio Transformer encoder
class AudioTransformer(nn.Module):
    def __init__(self, num_tokens=NUM_TOKENS, embed_dim=LATENT_DIM, depth=NUM_LAYERS, 
                 num_heads=NUM_HEADS, mlp_ratio=MLP_RATIO, drop_rate=0.1, num_classes=0):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim
        
        # Feature extractor (1D CNN)
        self.feature_extractor = CNNFeatureExtractor(embed_dim=embed_dim)
        
        # Position embeddings
        self.pos_embed = nn.Parameter(torch.zeros(1, num_tokens + 1, embed_dim))
        
        # Class token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        # Dropout
        self.pos_drop = nn.Dropout(p=drop_rate)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, 
                drop=drop_rate, attn_drop=drop_rate)
            for _ in range(depth)
        ])
        
        # Layer norm
        self.norm = nn.LayerNorm(embed_dim)
        
        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()
        
        # Initialize weights
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
    
    def forward_features(self, x, mask=None):
        # Extract CNN features: (B, C, L) -> (B, L, D)
        x = self.feature_extractor(x)
        
        # Add class token
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        
        # Add position embedding (with size check)
        # Ensure pos_embed size matches sequence length
        if x.size(1) > self.pos_embed.size(1):
            # If sequence is longer than position embeddings, truncate sequence
            x = x[:, :self.pos_embed.size(1), :]
        elif x.size(1) < self.pos_embed.size(1):
            # If sequence is shorter than position embeddings, truncate position embeddings
            pos_embed = self.pos_embed[:, :x.size(1), :]
            x = x + pos_embed
        else:
            # If sizes match, add directly
            x = x + self.pos_embed[:, :x.size(1)]
        x = self.pos_drop(x)
        
        # Apply transformer blocks
        for block in self.blocks:
            x = block(x, mask)
        
        # Apply final norm
        x = self.norm(x)
        return x
    
    def forward(self, x, mask=None):
        # Forward features
        x = self.forward_features(x, mask)
        
        # Use class token for classification
        if self.num_classes > 0:
            x = self.head(x[:, 0])
        return x

# Define the Transformer decoder block
class TransformerDecoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.self_attn = nn.MultiheadAttention(dim, num_heads, dropout=attn_drop, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.cross_attn = nn.MultiheadAttention(dim, num_heads, dropout=attn_drop, batch_first=True)
        self.norm3 = nn.LayerNorm(dim)
        self.mlp = MLP(in_features=dim, hidden_features=int(dim * mlp_ratio), drop=drop)
        
    def forward(self, x, context):
        # Self-attention
        self_attn_output, _ = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x))
        x = x + self_attn_output
        
        # Cross-attention
        cross_attn_output, _ = self.cross_attn(self.norm2(x), self.norm2(context), self.norm2(context))
        x = x + cross_attn_output
        
        # MLP
        x = x + self.mlp(self.norm3(x))
        return x

# Define the Transformer decoder
class TransformerDecoder(nn.Module):
    def __init__(self, embed_dim=LATENT_DIM, depth=NUM_DECODER_LAYERS, 
                 num_heads=NUM_HEADS, mlp_ratio=MLP_RATIO, drop_rate=0.1):
        super().__init__()
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerDecoderBlock(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, 
                drop=drop_rate, attn_drop=drop_rate)
            for _ in range(depth)
        ])
        
        # Layer norm
        self.norm = nn.LayerNorm(embed_dim)
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
    
    def forward(self, x, context):
        # Apply transformer blocks
        for block in self.blocks:
            x = block(x, context)
        
        # Apply final norm
        x = self.norm(x)
        return x

# Define the AJEPA model for 1D audio
class AJEPA1D(nn.Module):
    def __init__(self, num_tokens=NUM_TOKENS, embed_dim=LATENT_DIM, 
                 encoder_depth=NUM_LAYERS, decoder_depth=NUM_DECODER_LAYERS, 
                 num_heads=NUM_HEADS, mlp_ratio=MLP_RATIO, drop_rate=0.1,
                 num_classes=0):
        super().__init__()
        
        # Context encoder
        self.context_encoder = AudioTransformer(
            num_tokens=num_tokens, embed_dim=embed_dim, depth=encoder_depth, 
            num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=drop_rate, num_classes=0)
        
        # Target encoder (updated via EMA)
        self.target_encoder = AudioTransformer(
            num_tokens=num_tokens, embed_dim=embed_dim, depth=encoder_depth, 
            num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=drop_rate, num_classes=0)
        
        # Predictor (decoder)
        self.predictor = TransformerDecoder(
            embed_dim=embed_dim, depth=decoder_depth, 
            num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=drop_rate)
        
        # Classifier head (for fine-tuning)
        self.classifier = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()
        
        # Initialize target encoder with context encoder weights
        self._init_target_encoder()
        
    def _init_target_encoder(self):
        # Initialize target encoder with context encoder weights
        for param_target, param_context in zip(self.target_encoder.parameters(), self.context_encoder.parameters()):
            param_target.data.copy_(param_context.data)
            param_target.requires_grad = False
    
    def update_target_encoder(self, momentum=0.996):
        # Update target encoder via EMA
        for param_target, param_context in zip(self.target_encoder.parameters(), self.context_encoder.parameters()):
            param_target.data = momentum * param_target.data + (1 - momentum) * param_context.data
    
    def forward_pretrain(self, x, context_masks, target_masks):
        # Forward pass with context encoder
        context_features = self.context_encoder.forward_features(x, context_masks)
        
        # Forward pass with target encoder
        with torch.no_grad():
            target_features = self.target_encoder.forward_features(x, target_masks)
        
        # Forward pass with predictor
        pred_features = self.predictor(context_features, target_features)
        
        return pred_features, target_features
    
    def forward_finetune(self, x, mask=None):
        # Forward pass with context encoder
        features = self.context_encoder.forward_features(x, mask)
        
        # Extract [CLS] token for classification
        cls_token = features[:, 0]
        
        # Forward pass with classifier
        logits = self.classifier(cls_token)
        
        return logits

# Define mask generator functions for 1D audio signal
def generate_block_mask(seq_length, num_blocks=1, min_block_size=10, max_block_size=50):
    """Generate random block masks for 1D signal"""
    mask = torch.zeros(seq_length, dtype=torch.bool, device=device)
    
    for _ in range(num_blocks):
        # Sample block size
        block_size = torch.randint(min_block_size, max_block_size + 1, (1,)).item()
        
        # Sample start position
        start = torch.randint(0, seq_length - block_size + 1, (1,)).item()
        
        # Set mask
        mask[start:start + block_size] = True
    
    return mask

def generate_sparse_mask(seq_length, mask_ratio=0.2):
    """Generate sparse random masks for 1D signal"""
    num_masked = int(seq_length * mask_ratio)
    mask_indices = torch.randperm(seq_length, device=device)[:num_masked]
    
    mask = torch.zeros(seq_length, dtype=torch.bool, device=device)
    mask[mask_indices] = True
    
    return mask

def generate_contextual_mask(signal, seq_length, mask_ratio=0.5, energy_threshold=0.7):
    """Generate contextual masks based on signal energy"""
    # Compute energy
    if len(signal.shape) > 2:  # Handle batch dimension
        # Average across batch
        energy = torch.mean(torch.abs(signal), dim=0).squeeze()
    else:
        energy = torch.abs(signal).squeeze()
    
    # Normalize energy
    energy = energy / (energy.max() + 1e-6)
    
    # Downsample energy to match seq_length if needed
    if energy.shape[0] > seq_length:
        # Average pooling to downsample
        energy = F.avg_pool1d(energy.unsqueeze(0).unsqueeze(0), 
                              kernel_size=energy.shape[0] // seq_length,
                              stride=energy.shape[0] // seq_length).squeeze()
    
    # Create mask based on energy
    high_energy_regions = (energy > energy_threshold)
    
    # Ensure masking ratio is met
    if high_energy_regions.sum() < seq_length * mask_ratio:
        # Add more masks randomly if needed
        remaining = int(seq_length * mask_ratio) - high_energy_regions.sum().item()
        low_energy_indices = torch.where(~high_energy_regions)[0]
        if len(low_energy_indices) > 0:
            additional_indices = low_energy_indices[torch.randperm(len(low_energy_indices))[:remaining]]
            high_energy_regions[additional_indices] = True
    elif high_energy_regions.sum() > seq_length * mask_ratio:
        # Remove some masks if needed
        excess = high_energy_regions.sum().item() - int(seq_length * mask_ratio)
        high_energy_indices = torch.where(high_energy_regions)[0]
        remove_indices = high_energy_indices[torch.randperm(len(high_energy_indices))[:excess]]
        high_energy_regions[remove_indices] = False
    
    return high_energy_regions

def generate_curriculum_masks(batch_size, seq_length, inputs=None, curriculum_step=0.0):
    """
    Generate masks for curriculum learning.
    curriculum_step: 0.0 -> 1.0 from simple block to contextual masking.
    """
    context_masks = []
    target_masks = []
    
    for b in range(batch_size):
        # Determine masking stage based on curriculum_step
        if curriculum_step < 0.33:
            # Stage 1: Simple block masking (15-20% mask ratio)
            mask_ratio = 0.15 + 0.05 * curriculum_step / 0.33
            num_blocks = max(1, int(seq_length * mask_ratio / 30))
            target_mask = generate_block_mask(seq_length, num_blocks=num_blocks, 
                                            min_block_size=20, max_block_size=40)
            
        elif curriculum_step < 0.67:
            # Stage 2: Mixed masking (30-40% mask ratio)
            norm_step = (curriculum_step - 0.33) / 0.34
            mask_ratio = 0.3 + 0.1 * norm_step
            
            block_ratio = 0.7 - 0.4 * norm_step
            sparse_ratio = 1.0 - block_ratio
            
            block_mask = generate_block_mask(seq_length, 
                                            num_blocks=int(seq_length * mask_ratio * block_ratio / 30),
                                            min_block_size=15, max_block_size=35)
            sparse_mask = generate_sparse_mask(seq_length, mask_ratio=mask_ratio * sparse_ratio)
            
            # Move both masks to the same device before combining
            block_mask = block_mask.to(device)
            sparse_mask = sparse_mask.to(device)
            target_mask = block_mask | sparse_mask
            
        else:
            # Stage 3: Contextual-aware masking (50-60% mask ratio)
            norm_step = (curriculum_step - 0.67) / 0.33
            mask_ratio = 0.5 + 0.1 * norm_step
            
            if inputs is not None:
                # Use signal for contextual masking
                target_mask = generate_contextual_mask(inputs[b:b+1], seq_length, 
                                                    mask_ratio=mask_ratio,
                                                    energy_threshold=0.5)
            else:
                # Fallback to mixed masking if inputs not provided
                block_mask = generate_block_mask(seq_length, 
                                                num_blocks=int(seq_length * mask_ratio * 0.3 / 30),
                                                min_block_size=10, max_block_size=30)
                sparse_mask = generate_sparse_mask(seq_length, mask_ratio=mask_ratio * 0.7)
                
                # Move both masks to the same device before combining
                block_mask = block_mask.to(device)
                sparse_mask = sparse_mask.to(device)
                target_mask = block_mask | sparse_mask
        
        # Move target_mask to device if not already
        target_mask = target_mask.to(device)
        
        # Create complement mask for context (95% visible)
        context_visible_ratio = 0.95
        context_mask = generate_sparse_mask(seq_length, mask_ratio=(1.0 - context_visible_ratio))
        context_mask = context_mask.to(device)  # Move to the same device
        
        # Ensure context and target masks don't completely overlap
        overlap = context_mask & target_mask  # Now both are on the same device
        if overlap.sum() > 0.8 * target_mask.sum():
            # Too much overlap, regenerate context mask
            remaining_indices = torch.where(~target_mask)[0]
            if len(remaining_indices) > 0:
                num_context_masked = int(seq_length * (1.0 - context_visible_ratio))
                num_context_masked = min(num_context_masked, len(remaining_indices))
                
                context_indices = remaining_indices[torch.randperm(len(remaining_indices), device=device)[:num_context_masked]]
                context_mask = torch.zeros(seq_length, dtype=torch.bool, device=device)
                context_mask[context_indices] = True
        
        # Add padding for the [CLS] token (already on the correct device)
        context_attn_mask = torch.cat([torch.zeros((1), dtype=torch.bool, device=device), 
                                      context_mask], dim=0)
        target_attn_mask = torch.cat([torch.zeros((1), dtype=torch.bool, device=device), 
                                     target_mask], dim=0)
        
        context_masks.append(context_attn_mask)
        target_masks.append(target_attn_mask)
    
    # Stack masks (already on the correct device)
    context_masks = torch.stack(context_masks)
    target_masks = torch.stack(target_masks)
    
    return context_masks, target_masks

def generate_regularized_mask(batch_size, seq_length, mask_ratio=0.1):
    """
    Generate regularized mask for fine-tuning.
    """
    # Create masks
    masks = []
    
    for _ in range(batch_size):
        # Randomly select tokens to mask
        num_masked = int(seq_length * mask_ratio)
        mask_indices = torch.randperm(seq_length)[:num_masked]
        
        # Create mask
        mask = torch.zeros(seq_length, dtype=torch.bool)
        mask[mask_indices] = True
        
        # Add padding for the [CLS] token
        mask = torch.cat([torch.zeros((1), dtype=torch.bool), mask], dim=0)
        masks.append(mask)
    
    # Stack masks
    masks = torch.stack(masks).to(device)
    
    return masks

# Define the pretraining function
def pretrain(model, train_loader, num_epochs, optimizer, scheduler=None):
    model.train()
    
    # Loss function (cosine similarity loss)
    cos_sim = nn.CosineSimilarity(dim=-1)
    
    # Create gradient scaler for mixed precision
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Curriculum step (0 -> 1 over epochs)
        curriculum_step = min(1.0, epoch / (num_epochs * 0.8))
        
        # Progress bar
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for i, (inputs, _) in enumerate(progress_bar):
            inputs = inputs.to(device)
            
            # Generate masks
            context_masks, target_masks = generate_curriculum_masks(
                inputs.shape[0], NUM_TOKENS, inputs, curriculum_step=curriculum_step)
            
            # Forward pass with mixed precision
            with autocast():
                pred_features, target_features = model.forward_pretrain(inputs, context_masks, target_masks)
                
                # Calculate loss (negative cosine similarity)
                loss = -cos_sim(pred_features[:, 1:], target_features[:, 1:]).mean()
            
            # Backward pass and optimize with scaler
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Update target encoder
            model.update_target_encoder()
            
            # Update running loss
            running_loss += loss.item()
            
            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item()})
            
        # Print epoch loss
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        # Step scheduler
        if scheduler is not None:
            scheduler.step()
    
    return model

# Define evaluation function
def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    # Progress bar
    progress_bar = tqdm(test_loader, desc='Evaluating')
    
    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            
            # Generate regularized masks
            masks = generate_regularized_mask(inputs.shape[0], NUM_TOKENS, mask_ratio=MASK_RATIO_FINETUNE)
            
            # Forward pass
            outputs = model.forward_finetune(inputs, masks)
            
            # Store predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return accuracy, all_preds, all_labels

# Define the fine-tuning function
def finetune(model, train_loader, val_loader, num_epochs, optimizer, scheduler=None):
    best_val_acc = 0.0
    best_model_weights = None

    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    # To store metrics for plotting
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')

        for i, (inputs, labels) in enumerate(progress_bar):
            inputs = inputs.to(device)
            labels = labels.to(device)

            masks = generate_regularized_mask(inputs.shape[0], NUM_TOKENS, mask_ratio=MASK_RATIO_FINETUNE)

            with autocast():
                outputs = model.forward_finetune(inputs, masks)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix({'loss': loss.item()})

        train_loss = running_loss / len(train_loader)
        train_acc = accuracy_score(all_labels, all_preds)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        model.eval()
        val_running_loss = 0.0
        all_val_preds = []
        all_val_labels = []

        val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_progress_bar):
                inputs = inputs.to(device)
                labels = labels.to(device)

                masks = generate_regularized_mask(inputs.shape[0], NUM_TOKENS, mask_ratio=MASK_RATIO_FINETUNE)

                outputs = model.forward_finetune(inputs, masks)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_val_preds.extend(preds.cpu().numpy())
                all_val_labels.extend(labels.cpu().numpy())

                val_progress_bar.set_postfix({'loss': loss.item()})

        val_loss = val_running_loss / len(val_loader)
        val_acc = accuracy_score(all_val_labels, all_val_preds)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_weights = model.state_dict().copy()
            print(f'  New best model! Val Acc: {val_acc:.4f}')

        if scheduler is not None:
            scheduler.step()

    model.load_state_dict(best_model_weights)
    print(f'Best validation accuracy: {best_val_acc:.4f}')

    # Plot Learning Curves
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Epoch vs Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Epoch vs Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return model

# Function to visualize masking
def visualize_masking(waveform, mask, title="Masked Audio"):
    """Visualize the original waveform and the masked regions."""
    plt.figure(figsize=(12, 6))
    
    # Plot original waveform
    plt.subplot(2, 1, 1)
    plt.plot(waveform.squeeze().cpu().numpy())
    plt.title("Original Waveform")
    plt.ylabel("Amplitude")
    
    # Create mask visualization
    plt.subplot(2, 1, 2)
    
    # Upsample mask to match waveform length if needed
    if len(mask) != waveform.shape[1]:
        binary_mask = torch.zeros(waveform.shape[1])
        mask_len = len(mask) - 1  # Account for CLS token
        step = waveform.shape[1] // mask_len
        
        for i in range(mask_len):
            if mask[i+1]:  # Skip CLS token
                binary_mask[i*step:(i+1)*step] = 1
    else:
        binary_mask = mask
    
    masked_waveform = waveform.clone().squeeze().cpu().numpy()
    mask_indices = binary_mask.bool().cpu().numpy()
    masked_waveform[mask_indices] = np.nan  # Set masked regions to NaN
    
    plt.plot(masked_waveform)
    plt.title(f"{title} (masked regions removed)")
    plt.ylabel("Amplitude")
    plt.xlabel("Sample")
    
    plt.tight_layout()
    plt.show()

def visualize_masking(waveform, mask, title):
    """Visualize the original waveform and the masked version."""
    plt.figure(figsize=(12, 4))
    
    # Ensure dimensions match before masking
    waveform_np = waveform.clone().squeeze().cpu().numpy()
    mask_np = mask.squeeze().cpu().numpy()
    
    # Make sure mask is the same length as waveform
    if len(mask_np) != len(waveform_np):
        # Resize mask to match waveform length
        if len(mask_np) == 1:  # Special case: singleton mask
            mask_np = np.repeat(mask_np, len(waveform_np))
        else:
            # Resample mask to match waveform length
            indices = np.round(np.linspace(0, len(mask_np) - 1, len(waveform_np))).astype(int)
            mask_np = mask_np[indices]
    
    # Create masked waveform
    masked_waveform = waveform_np.copy()
    masked_waveform[mask_np > 0.5] = np.nan  # Set masked regions to NaN
    
    plt.plot(masked_waveform)
    plt.title(title)
    plt.xlabel("Sample")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

# Main execution
def main():
    print("Starting A-JEPA with 1D CNN implementation for Environment Sound Classification")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        print("CUDA cache cleared.")
    else:
        print("CUDA is not available.")
    
    # Load full dataset and update class list
    global ENV_SOUND_CLASSES, NUM_CLASSES
    
    # Check if the dataset path exists
    if not os.path.exists(DATA_PATH):
        print(f"Error: Dataset path {DATA_PATH} does not exist.")
        print("Available paths:")
        for path in os.listdir('/kaggle/input'):
            print(f" - /kaggle/input/{path}")
        return
    
    # Get the actual class names from the directory
    ENV_SOUND_CLASSES = sorted([d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))])
    NUM_CLASSES = len(ENV_SOUND_CLASSES)
    
    print(f"Found {NUM_CLASSES} classes: {ENV_SOUND_CLASSES}")
    
    # Load dataset
    full_dataset = EnvironmentSoundDataset(DATA_PATH)
    print(f'Dataset loaded, size: {len(full_dataset)}')
    
    # Train/Validation split
    indices = list(range(len(full_dataset)))
    train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42, stratify=full_dataset.labels)

    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)

    # Create data loaders (use num_workers=0 if having issues with parallelism in Kaggle)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Create model
    model = AJEPA1D(
        num_tokens=NUM_TOKENS,
        embed_dim=LATENT_DIM,
        encoder_depth=NUM_LAYERS,
        decoder_depth=NUM_DECODER_LAYERS,
        num_heads=NUM_HEADS,
        num_classes=NUM_CLASSES,
        mlp_ratio=MLP_RATIO,
        drop_rate=0.1
    ).to(device)
    
    print(f"Model created with parameters:")
    print(f"- Tokens: {NUM_TOKENS}")
    print(f"- Sample rate: {SAMPLE_RATE}")
    print(f"- CNN kernel size: {CNN_KERNEL_SIZE} ({CNN_KERNEL_SIZE/SAMPLE_RATE*1000:.1f}ms)")
    print(f"- CNN stride: {CNN_STRIDE} ({CNN_STRIDE/SAMPLE_RATE*1000:.1f}ms)")
    print(f"- Latent dimension: {LATENT_DIM}")
    print(f"- Encoder layers: {NUM_LAYERS}")
    print(f"- Decoder layers: {NUM_DECODER_LAYERS}")
    print(f"- Attention heads: {NUM_HEADS}")
    print(f"- Classes: {NUM_CLASSES} ({ENV_SOUND_CLASSES})")
    
    # --- Pretraining ---
    print("\n=== Starting Pretraining ===")
    
    # Create optimizer for pretraining
    pretrain_optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE_PRETRAIN,
        weight_decay=WEIGHT_DECAY
    )
    
    # Create learning rate scheduler
    pretrain_scheduler = optim.lr_scheduler.CosineAnnealingLR(
        pretrain_optimizer,
        T_max=NUM_EPOCHS_PRETRAIN
    )
    
    # Pretrain model
    model = pretrain(
        model,
        train_loader,
        NUM_EPOCHS_PRETRAIN,
        pretrain_optimizer,
        pretrain_scheduler
    )
    
    # Save pretrained model checkpoint
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': pretrain_optimizer.state_dict(),
    }, 'ajepa_1d_pretrained.pth')
    
    print("Pretraining completed and checkpoint saved.")
    
    # --- Fine-tuning ---
    print("\n=== Starting Fine-tuning ===")
    
    # Create optimizer for fine-tuning
    finetune_optimizer = optim.AdamW(
        [
            {'params': model.context_encoder.parameters()},
            {'params': model.classifier.parameters(), 'lr': LEARNING_RATE_FINETUNE * 10}
        ],
        lr=LEARNING_RATE_FINETUNE,
        weight_decay=WEIGHT_DECAY
    )
    
    # Create learning rate scheduler
    finetune_scheduler = optim.lr_scheduler.CosineAnnealingLR(
        finetune_optimizer,
        T_max=NUM_EPOCHS_FINETUNE
    )
    
    # Fine-tune model
    model = finetune(
        model,
        train_loader,
        val_loader,
        NUM_EPOCHS_FINETUNE,
        finetune_optimizer,
        finetune_scheduler
    )
    
    # Save fine-tuned model checkpoint
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': finetune_optimizer.state_dict(),
    }, 'ajepa_1d_finetuned.pth')
    
    print("Fine-tuning completed and checkpoint saved.")
    
    # --- Evaluation ---
    print("\n=== Evaluating Model ===")
    
    # Evaluate model
    test_accuracy, test_preds, test_labels = evaluate(model, val_loader)
    
    # Visualize confusion matrix
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    
    cm = confusion_matrix(test_labels, test_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=ENV_SOUND_CLASSES, yticklabels=ENV_SOUND_CLASSES)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.show()
    
    print(f"Final Test Accuracy: {test_accuracy:.4f}")
    print("Confusion matrix saved as 'confusion_matrix.png'")
        
    # Main visualization code
    print("\n=== Visualizing Masking Strategies ===")
        
    # Get a sample from the validation set
    sample_inputs, _ = next(iter(val_loader))
    sample_input = sample_inputs[0:1]  # Take first sample
        
    # Generate masks for different curriculum stages
    for curriculum_step, stage_name in [(0.1, "Stage 1 (Easy)"), 
                                      (0.5, "Stage 2 (Medium)"), 
                                      (0.9, "Stage 3 (Hard)")]:
        # Generate masks
        __, target_masks = generate_curriculum_masks(
            1, NUM_TOKENS, sample_input, curriculum_step=curriculum_step)
        
        # Pass directly to the updated visualize_masking function
        # which now handles size mismatches internally
        visualize_masking(sample_input, target_masks[0], title=f"Masking: {stage_name}")
        
    print("Masking visualization completed.")
    
# Run the main function
if __name__ == "__main__":
    main()

Using device: cuda
Starting A-JEPA with 1D CNN implementation for Environment Sound Classification
CUDA cache cleared.
Error: Dataset path /kaggle/input/environment-sound/Environment_Sound does not exist.
Available paths:
